In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee

In [ ]:
ee.Initialize()
# ! earthengine authenticate

In [ ]:
datasets = "/home/sherwood/learning/Data4Policy/Geospatial Data Science Internship/Datasets/"


telangana_fires = gpd.read_file(datasets + 'telangana_fires.csv')

telangana_fires = gpd.GeoDataFrame(telangana_fires,
                                   geometry = gpd.points_from_xy(
                                       telangana_fires.longitude,
                                       telangana_fires.latitude),
                                    crs = 'EPSG:4326'
                                   )

telangana_fires

In [31]:
# Defining the map
Map = geemap.Map()
# Map

In [ ]:
# Getting the path from the Telengana shapefile from GADM
telangana_path = '/mnt/c/Users/amoam/Downloads/gadm40_IND_shp/telangana_shape.shp'
telangana_shp = geemap.shp_to_ee(telangana_path)
center = telangana_shp.centroid().getInfo()['coordinates']
center.reverse()

In [ ]:
# ee.Geometry accepts Geojson
geometry = ee.Geometry.Polygon('/mnt/c/Users/amoam/Downloads/telangana_shapefile.geojson')
# no funka

In [28]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")\
                            .map( lambda image: image.clip(telangana_shp))
                    

In [52]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': esri_lulc10,
    'description': 'imageToDriveExample',
    'folder':'Example_folder',
    'region': telangana_shp
})
task.start()

In [53]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).
Polling for task (id: FQC37447QNH5AZIJCLSMKRVB).


In [41]:
image_viz_params = {
    "names": [
        "Water",
        "Trees",
        "Grass",
        "Flooded Vegetation",
        "Crops",
        "Scrub/Shrub",
        "Built Area",
        "Bare Ground",
        "Snow/Ice",
        "Clouds"
    ],
    "colors": [
        "#1A5BAB",
        "#358221",
        "#A7D282",
        "#87D19E",
        "#FFDB5C",
        "#EECFA8",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8"
    ]
}


In [42]:
Map.addLayer(telangana_shp, {}, "Telangana")
Map.addLayer(esri_lulc10, { 'min':1, 'max':10, 'palette':image_viz_params['colors'] }, "ESRI-LULC-MAP")
Map.centerObject(telangana_shp, 7)
Map

Map(bottom=15038.0, center=[17.792484085556335, 79.05826443037053], controls=(WidgetControl(options=['position…

## Visualization

In [ ]:
Map = geemap.Map(center=[17.123184, 79.208824], zoom=7)
Map

In [ ]:
file_path= os.path.abspath('telangana_shapefile.geojson')

with open(file_path) as f:
    json_data = json.load(f)



In [ ]:
json_layer = GeoJSON(
    data=json_data,
    name='Telangana',
    hover_style={'fillColor':'blue','fillOpacity':0.4},
)

Map.add_layer(json_layer)


ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data,{})


js_data_2 = ee_to_geojson(ee_data)
js_layer_2 = GeoJSON(
    data = js_data_2,
    name = 'Telengana EE JSON',
    hover_style = {'fillColor':'red','fillOpacity':0.5},
)
Map.add_layer(js_layer_2)

In [ ]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")#\
                            #.map( lambda image: image.clip(js_layer_2))

# Define a dictionary which will be used to make visualize image on map
dict = {
  "names": [
    "Water",
    "Trees",
    "Grass",
    "Flooded Vegetation",
    "Crops",
    "Scrub/Shrub",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#A7D282",
    "#87D19E",
    "#FFDB5C",
    "#EECFA8",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8"
  ]}


Map.addLayer(esri_lulc10.mosaic(), {min:1, max:10, palette:dict['colors']})